https://www.kaggle.com/datasets/bitext/training-dataset-for-chatbotsvirtual-assistants

In [1]:
import pandas as pd
from textblob import TextBlob
import nltk
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Conv1D,MaxPooling1D,GlobalMaxPooling1D,Flatten,SimpleRNN
from keras import regularizers
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing


In [16]:
df=pd.read_csv("D:\\documents\\internship\\rasachatbot\intent classification\\20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample\\20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample\\training.csv")
df=df.iloc[:,1::2]
df

,utterance,intent
0,"I don't have an online account, what do I have...",create_account
1,can you tell me if i can regisger two accounts...,create_account
2,"I have no online account, open one, please",create_account
3,"could you ask an agent how to open an account,...",create_account
4,"i want an online account, create one",create_account
...,...,...
21529,"i have no shipping address, what do i have to ...",set_up_shipping_address
21530,I have no shipping address and I want to set o...,set_up_shipping_address
21531,"i want to set up my shipping address, what do ...",set_up_shipping_address
21532,"I don't have a shipping address, can you set o...",set_up_shipping_address


In [17]:
max_words = 5000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['utterance'].values)
sequences = tokenizer.texts_to_sequences(df['utterance'].values)
userinputs = pad_sequences(sequences, maxlen=max_len)

In [19]:
df["intent"]=preprocessing.LabelEncoder().fit_transform(df["intent"])
xtrain=userinputs
ytrain=df["intent"]
ytrain = to_categorical(ytrain,30)


In [28]:
dftest=pd.read_csv("D:\\documents\\internship\\rasachatbot\\intent classification\\Bitext_Sample_Customer_Service_Testing_Dataset.csv")
dftest=dftest.iloc[:,:2]
tokenizer.fit_on_texts(dftest['utterance'].values)
sequences = tokenizer.texts_to_sequences(dftest['utterance'].values)
userinputstest= pad_sequences(sequences, maxlen=max_len)
dftest.head()

,utterance,intent
0,I have a question about cancelling an order,cancel_order
1,help canceling the order I have made,cancel_order
2,I do not know how to cancel an order I have made,cancel_order
3,where can I cancel the last order I made?,cancel_order
4,I don't want the last order I made,cancel_order


In [26]:
print(len(df),len(dftest))

21534 21534


In [29]:
xtest=userinputstest
dftest["intent"]=LabelEncoder().fit_transform(dftest["intent"])
ytest=dftest["intent"]
ytest = to_categorical(ytest,30)

In [23]:
from keras import layers
model2 = Sequential()
model2.add(layers.Embedding(max_words, 32, input_length=max_len)) 
model2.add(layers.Bidirectional(layers.LSTM(15,dropout=0.5))) 
model2.add(layers.Dense(30,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model2.fit(xtrain, ytrain, epochs=20)
model2.evaluate(xtest,ytest)

Epoch 1/20
673/673 [==============================] - 41s 61ms/step - loss: 1.8727 - accuracy: 0.5326
Epoch 2/20
673/673 [==============================] - 41s 61ms/step - loss: 0.6616 - accuracy: 0.8577
Epoch 3/20
673/673 [==============================] - 41s 61ms/step - loss: 0.3243 - accuracy: 0.9233
Epoch 4/20
673/673 [==============================] - 42s 62ms/step - loss: 0.1996 - accuracy: 0.9525
Epoch 5/20
673/673 [==============================] - 41s 61ms/step - loss: 0.1379 - accuracy: 0.9703
Epoch 6/20
673/673 [==============================] - 41s 61ms/step - loss: 0.1013 - accuracy: 0.9777
Epoch 7/20
673/673 [==============================] - 41s 61ms/step - loss: 0.0766 - accuracy: 0.9839
Epoch 8/20
673/673 [==============================] - 41s 61ms/step - loss: 0.0605 - accuracy: 0.9869
Epoch 9/20
673/673 [==============================] - 42s 62ms/step - loss: 0.0515 - accuracy: 0.9876
Epoch 10/20
673/673 [==============================] - 41s 61ms/step - loss: 0.042

[0.005426750052720308, 0.9984675645828247]

In [32]:
model2.evaluate(xtest,ytest)

26/26 [==============================] - 0s 16ms/step - loss: 6.5688 - accuracy: 0.1650


[6.568804740905762, 0.16503667831420898]